In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## DO NOT RUN BUT JUST TO SHOW HOW WE GOT COMBINED FILE!

In [ ]:
# # Test with one game
# GAME_ID_TEST = 2023110500

# # Filter both dataframes for the test game and players to predict
# pre_game_df = pre_df[
#     (pre_df['game_id'] == GAME_ID_TEST) & 
#     (pre_df['player_to_predict'] == True)
# ].copy()

# post_game_df = post_df[
#     (post_df['game_id'] == GAME_ID_TEST)
#     # (post_df['player_to_predict'] == True)
# ].copy()

# print(f"Pre-game rows: {len(pre_game_df)}")
# print(f"Post-game rows: {len(post_game_df)}")

# # 1. Create game_play column in pre_df
# pre_game_df['game_play'] = pre_game_df['game_id'].astype(str) + '_' + pre_game_df['play_id'].astype(str)

# # 2. Group by game_play and find max frame_id
# game_play_offset_df = pre_game_df.groupby('game_play')['frame_id'].max().reset_index()
# game_play_offset_df.columns = ['game_play', 'max_frame_id']

# print(f"\nUnique plays in game: {len(game_play_offset_df)}")
# print(f"Sample offsets:\n{game_play_offset_df.head()}")

# # 3. Create game_play column in post_df
# post_game_df['game_play'] = post_game_df['game_id'].astype(str) + '_' + post_game_df['play_id'].astype(str)

# # 4. Join offset df onto post_df
# post_game_df = post_game_df.merge(game_play_offset_df, on='game_play', how='left')

# # 5. Calculate new frame_id (total continuous frame_id)
# post_game_df['frame_id_total'] = post_game_df['max_frame_id'] + post_game_df['frame_id']

# # 6. Prepare pre_df for concatenation
# pre_game_df['frame_id_total'] = pre_game_df['frame_id']

# # Add 'frame' column to identify pre vs post
# pre_game_df['frame'] = 'pre'
# post_game_df['frame'] = 'post'

# # Drop temporary columns before concat
# post_game_df = post_game_df.drop(columns=['max_frame_id', 'frame_id'], errors='ignore')
# pre_game_df = pre_game_df.drop(columns=['frame_id'], errors='ignore')

# # 7. Concatenate pre and post together
# df_combined = pd.concat([pre_game_df, post_game_df], ignore_index=True)

# print(f"\n{'='*50}")
# print(f"Total combined rows: {len(df_combined)}")
# print(f"Unique plays: {df_combined['game_play'].nunique()}")
# print(f"Unique players: {df_combined['nfl_id'].nunique()}")
# print(f"\nFrame ID range: {df_combined['frame_id_total'].min()} to {df_combined['frame_id_total'].max()}")
# print(f"Pre rows: {(df_combined['frame'] == 'pre').sum()}")
# print(f"Post rows: {(df_combined['frame'] == 'post').sum()}")

# # Verify continuity for one player in one play
# sample_player = df_combined.groupby(['game_play', 'nfl_id']).size().reset_index().iloc[0]
# sample_check = df_combined[
#     (df_combined['game_play'] == sample_player['game_play']) & 
#     (df_combined['nfl_id'] == sample_player['nfl_id'])
# ].sort_values('frame_id_total')

# print(f"\nSample verification (Play: {sample_player['game_play']}, Player: {sample_player['nfl_id']}):")
# print(sample_check[['game_play', 'nfl_id', 'frame_id_total', 'frame']].head())
# print("...")
# print(sample_check[['game_play', 'nfl_id', 'frame_id_total', 'frame']].tail())

# # Sort by game_play, nfl_id, and frame_id_total to ensure proper order
# df_combined = df_combined.sort_values(['game_play', 'nfl_id', 'frame_id_total']).reset_index(drop=True)

In [14]:
df_combined = pd.read_csv(r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\combined_2023_w09_game_2023110500.csv")
df_combined.head()

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,...,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,game_play,frame_id_total,frame
0,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.04,0.41,343.71,66.02,18.0,74.639999,4.99,2023110500_1102,1,pre
1,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.10,0.51,2.28,69.18,18.0,74.639999,4.99,2023110500_1102,2,pre
2,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.32,1.10,0.92,73.59,18.0,74.639999,4.99,2023110500_1102,3,pre
3,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.43,1.27,1.79,76.49,18.0,74.639999,4.99,2023110500_1102,4,pre
4,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.84,2.09,0.19,83.07,18.0,74.639999,4.99,2023110500_1102,5,pre


## Avoiding Time Lagging for Neural Network
- we decided instead of including time lagging into our model, we could just use wide format and include where the player will go next in that row. then in our neural network we can just say predict the post frames for each nfl_id, play_id in each game (here we only have 1 game) by using X amount of explanantory variabels (s,a,o?,dir,ball_land_x, ball_land_y, etc). The specific number of rows to predict is in num_frames_output which is neccesary to tell the model because the test data only has input (pre release data) and that # changes for each play. 
### first we start with 1 player to verify format

In [16]:
# Filter for just Kader Kohou on play 55
kader_df = df_combined[
    (df_combined['nfl_id'] == 54936) & 
    (df_combined['play_id'] == 55)
].copy()

# Sort to ensure proper order
kader_df = kader_df.sort_values('frame_id_total').reset_index(drop=True)

# Create x_next and y_next by shifting x and y up by 1 row
kader_df['x_next'] = kader_df['x'].shift(-1)
kader_df['y_next'] = kader_df['y'].shift(-1)

print(f"Total rows for Kader Kohou (Play 55): {len(kader_df)}")
print(f"\nFirst few rows:")
print(kader_df[['frame_id_total', 'frame', 'x', 'y', 'x_next', 'y_next']].head(10))
print(f"\nLast few rows (note NaN in last row):")
print(kader_df[['frame_id_total', 'frame', 'x', 'y', 'x_next', 'y_next']].tail(5))

kader_df

Total rows for Kader Kohou (Play 55): 51

First few rows:
   frame_id_total frame      x      y  x_next  y_next
0               1   pre  39.89  23.94   39.91   24.05
1               2   pre  39.91  24.05   39.93   24.12
2               3   pre  39.93  24.12   39.94   24.18
3               4   pre  39.94  24.18   39.95   24.23
4               5   pre  39.95  24.23   39.96   24.25
5               6   pre  39.96  24.25   39.98   24.28
6               7   pre  39.98  24.28   40.00   24.30
7               8   pre  40.00  24.30   40.02   24.33
8               9   pre  40.02  24.33   40.04   24.38
9              10   pre  40.04  24.38   40.07   24.44

Last few rows (note NaN in last row):
    frame_id_total frame      x      y  x_next  y_next
46              47  post  50.78  27.48   51.22   28.02
47              48  post  51.22  28.02   51.65   28.59
48              49  post  51.65  28.59   52.07   29.19
49              50  post  52.07  29.19   52.50   29.82
50              51  post  52.50  2

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,...,dir,o,num_frames_output,ball_land_x,ball_land_y,game_play,frame_id_total,frame,x_next,y_next
0,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,10.24,294.41,13.0,50.720001,39.790001,2023110500_55,1,pre,39.91,24.05
1,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,10.04,293.17,13.0,50.720001,39.790001,2023110500_55,2,pre,39.93,24.12
2,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,11.75,291.37,13.0,50.720001,39.790001,2023110500_55,3,pre,39.94,24.18
3,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,13.26,288.75,13.0,50.720001,39.790001,2023110500_55,4,pre,39.95,24.23
4,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,16.62,286.05,13.0,50.720001,39.790001,2023110500_55,5,pre,39.96,24.25
5,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,28.95,283.16,13.0,50.720001,39.790001,2023110500_55,6,pre,39.98,24.28
6,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,47.74,280.29,13.0,50.720001,39.790001,2023110500_55,7,pre,40.00,24.30
7,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,40.13,279.65,13.0,50.720001,39.790001,2023110500_55,8,pre,40.02,24.33
8,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,33.90,279.65,13.0,50.720001,39.790001,2023110500_55,9,pre,40.04,24.38
9,2023110500,55,True,54936,right,35.0,Kader Kohou,5-9,187.0,1998-11-28,...,29.61,279.65,13.0,50.720001,39.790001,2023110500_55,10,pre,40.07,24.44


## now do it for the entire game

In [11]:
GAME_ID_TEST = 2023110500
# # Take the entire game's combined dataframe
df_game_with_next = df_combined.copy()

# Sort by game_play, nfl_id, and frame_id_total to ensure proper order
df_game_with_next = df_game_with_next.sort_values(['game_play', 'nfl_id', 'frame_id_total']).reset_index(drop=True)

# Create x_next and y_next by shifting within each player's trajectory
# We use groupby to ensure we only look ahead within the same player on the same play
df_game_with_next['x_next'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['x'].shift(-1)
df_game_with_next['y_next'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['y'].shift(-1)

print(f"Total rows: {len(df_game_with_next)}")
print(f"Unique plays: {df_game_with_next['game_play'].nunique()}")
print(f"Unique players: {df_game_with_next['nfl_id'].nunique()}")

# Verify with Kader Kohou on play 55
kader_check = df_game_with_next[
    (df_game_with_next['nfl_id'] == 54936) & 
    (df_game_with_next['play_id'] == 55)
]
print(f"\nKader Kohou (Play 55) - First few rows:")
print(kader_check[['frame_id_total', 'frame', 'x', 'x_next', 'y', 'y_next']].head(10))
print(f"\nKader Kohou (Play 55) - Last few rows:")
print(kader_check[['frame_id_total', 'frame', 'x', 'x_next', 'y', 'y_next']].tail(5))

# # Save the updated dataframe
output_path = r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_{}.csv".format(GAME_ID_TEST)
df_game_with_next.to_csv(output_path, index=False)

print(f"\n✓ Saved with x_next and y_next columns to: {output_path}")

df_game_with_next

Total rows: 6529
Unique plays: 47
Unique players: 39

Kader Kohou (Play 55) - First few rows:
      frame_id_total frame      x  x_next      y  y_next
5704               1   pre  39.89   39.91  23.94   24.05
5705               2   pre  39.91   39.93  24.05   24.12
5706               3   pre  39.93   39.94  24.12   24.18
5707               4   pre  39.94   39.95  24.18   24.23
5708               5   pre  39.95   39.96  24.23   24.25
5709               6   pre  39.96   39.98  24.25   24.28
5710               7   pre  39.98   40.00  24.28   24.30
5711               8   pre  40.00   40.02  24.30   24.33
5712               9   pre  40.02   40.04  24.33   24.38
5713              10   pre  40.04   40.07  24.38   24.44

Kader Kohou (Play 55) - Last few rows:
      frame_id_total frame      x  x_next      y  y_next
5750              47  post  50.78   51.22  27.48   28.02
5751              48  post  51.22   51.65  28.02   28.59
5752              49  post  51.65   52.07  28.59   29.19
5753       

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,...,dir,o,num_frames_output,ball_land_x,ball_land_y,game_play,frame_id_total,frame,x_next,y_next
0,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,343.71,66.02,18.0,74.639999,4.99,2023110500_1102,1,pre,49.24,12.43
1,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,2.28,69.18,18.0,74.639999,4.99,2023110500_1102,2,pre,49.24,12.46
2,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.92,73.59,18.0,74.639999,4.99,2023110500_1102,3,pre,49.24,12.49
3,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,1.79,76.49,18.0,74.639999,4.99,2023110500_1102,4,pre,49.24,12.57
4,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,0.19,83.07,18.0,74.639999,4.99,2023110500_1102,5,pre,49.24,12.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6524,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023110500_978,56,post,54.60,13.61
6525,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023110500_978,57,post,54.96,13.45
6526,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023110500_978,58,post,55.32,13.30
6527,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023110500_978,59,post,55.67,13.18


In [18]:
GAME_ID_TEST = 2023110500

# Take the entire game's combined dataframe
df_game_with_lags = df_combined.copy()

# Sort by game_play, nfl_id, and frame_id_total to ensure proper order
df_game_with_lags = df_game_with_lags.sort_values(['game_play', 'nfl_id', 'frame_id_total']).reset_index(drop=True)

# Create all lag/lead columns at once
df_game_with_lags['x_before'] = df_game_with_lags.groupby(['game_play', 'nfl_id'])['x'].shift(1)
df_game_with_lags['y_before'] = df_game_with_lags.groupby(['game_play', 'nfl_id'])['y'].shift(1)
df_game_with_lags['x_next'] = df_game_with_lags.groupby(['game_play', 'nfl_id'])['x'].shift(-1)
df_game_with_lags['y_next'] = df_game_with_lags.groupby(['game_play', 'nfl_id'])['y'].shift(-1)

print(f"Total rows: {len(df_game_with_lags)}")
print(f"Unique plays: {df_game_with_lags['game_play'].nunique()}")
print(f"Unique players: {df_game_with_lags['nfl_id'].nunique()}")

# Verify with Kader Kohou on play 55
kader_check = df_game_with_lags[
    (df_game_with_lags['nfl_id'] == 54936) & 
    (df_game_with_lags['play_id'] == 55)
]
print(f"\nKader Kohou (Play 55) - First few rows:")
print(kader_check[['frame_id_total', 'frame', 'x_before', 'x', 'x_next', 'y_before', 'y', 'y_next']].head(10))
print(f"\nKader Kohou (Play 55) - Last few rows:")
print(kader_check[['frame_id_total', 'frame', 'x_before', 'x', 'x_next', 'y_before', 'y', 'y_next']].tail(5))

# Save the updated dataframe
output_path = r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_{}.csv".format(GAME_ID_TEST)
df_game_with_lags.to_csv(output_path, index=False)

print(f"\n✓ Saved with x_before, y_before, x_next, and y_next columns to: {output_path}")

df_game_with_lags

Total rows: 6529
Unique plays: 47
Unique players: 39

Kader Kohou (Play 55) - First few rows:
      frame_id_total frame  x_before      x  x_next  y_before      y  y_next
5704               1   pre       NaN  39.89   39.91       NaN  23.94   24.05
5705               2   pre     39.89  39.91   39.93     23.94  24.05   24.12
5706               3   pre     39.91  39.93   39.94     24.05  24.12   24.18
5707               4   pre     39.93  39.94   39.95     24.12  24.18   24.23
5708               5   pre     39.94  39.95   39.96     24.18  24.23   24.25
5709               6   pre     39.95  39.96   39.98     24.23  24.25   24.28
5710               7   pre     39.96  39.98   40.00     24.25  24.28   24.30
5711               8   pre     39.98  40.00   40.02     24.28  24.30   24.33
5712               9   pre     40.00  40.02   40.04     24.30  24.33   24.38
5713              10   pre     40.02  40.04   40.07     24.33  24.38   24.44

Kader Kohou (Play 55) - Last few rows:
      frame_id_tota

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,...,num_frames_output,ball_land_x,ball_land_y,game_play,frame_id_total,frame,x_before,y_before,x_next,y_next
0,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,1,pre,NaN,NaN,49.24,12.43
1,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,2,pre,49.24,12.42,49.24,12.46
2,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,3,pre,49.24,12.43,49.24,12.49
3,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,4,pre,49.24,12.46,49.24,12.57
4,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,5,pre,49.24,12.49,49.24,12.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6524,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2023110500_978,56,post,53.90,14.04,54.60,13.61
6525,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2023110500_978,57,post,54.25,13.81,54.96,13.45
6526,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2023110500_978,58,post,54.60,13.61,55.32,13.30
6527,2023110500,978,NaN,46259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2023110500_978,59,post,54.96,13.45,55.67,13.18


## create x_before and y_before

In [ ]:
GAME_ID_TEST = 2023110500
# Take the entire game's combined dataframe
df_game_with_next = df_combined.copy()

# Sort by game_play, nfl_id, and frame_id_total to ensure proper order
df_game_with_next = df_game_with_next.sort_values(['game_play', 'nfl_id', 'frame_id_total']).reset_index(drop=True)

# Create x_next and y_next by shifting within each player's trajectory
# We use groupby to ensure we only look ahead within the same player on the same play
df_game_with_next['x_next'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['x'].shift(-1)
df_game_with_next['y_next'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['y'].shift(-1)

# Create x_before and y_before by shifting in the opposite direction (positive shift)
df_game_with_next['x_before'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['x'].shift(1)
df_game_with_next['y_before'] = df_game_with_next.groupby(['game_play', 'nfl_id'])['y'].shift(1)

print(f"Total rows: {len(df_game_with_next)}")
print(f"Unique plays: {df_game_with_next['game_play'].nunique()}")
print(f"Unique players: {df_game_with_next['nfl_id'].nunique()}")

# Verify with Kader Kohou on play 55
kader_check = df_game_with_next[
    (df_game_with_next['nfl_id'] == 54936) & 
    (df_game_with_next['play_id'] == 55)
]
print(f"\nKader Kohou (Play 55) - First few rows:")
print(kader_check[['frame_id_total', 'frame', 'x_before', 'x', 'x_next', 'y_before', 'y', 'y_next']].head(10))
print(f"\nKader Kohou (Play 55) - Last few rows:")
print(kader_check[['frame_id_total', 'frame', 'x_before', 'x', 'x_next', 'y_before', 'y', 'y_next']].tail(5))

# Save the updated dataframe
output_path = r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_{}.csv".format(GAME_ID_TEST)
df_game_with_next.to_csv(output_path, index=False)

print(f"\n✓ Saved with x_next, y_next, x_before, and y_before columns to: {output_path}")

df_game_with_next

In [19]:
df_lagged = pd.read_csv(r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_2023110500.csv")
df_lagged.head()

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,...,num_frames_output,ball_land_x,ball_land_y,game_play,frame_id_total,frame,x_before,y_before,x_next,y_next
0,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,1,pre,NaN,NaN,49.24,12.43
1,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,2,pre,49.24,12.42,49.24,12.46
2,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,3,pre,49.24,12.43,49.24,12.49
3,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,4,pre,49.24,12.46,49.24,12.57
4,2023110500,1102,True,43454,right,51.0,Tyreek Hill,5-10,185.0,1994-03-01,...,18.0,74.639999,4.99,2023110500_1102,5,pre,49.24,12.49,49.24,12.67


## Lagged data application to neural network
- read in "df_lagged = pd.read_csv(r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_2023110500.csv")
df_lagged.head()"
- format test data the with x_next, y_next, x_before, y_before the same way we did with df_lagged (file path of test = "C:\Users\Jenna\Downloads\nfl-big-data-bowl-2026-prediction\test_input.csv")
- with huge lagged df for 1 game (with x_before and x_after along with y), we will train a very simple neural network based on the 1 game (know it is important to include more games but we will do this right now). In that our pseudo code will follow: predict the post frames (identified for specific # in num_frames_output) for each nfl_id, play_id in each game (here we only have 1 game) by using X amount of explanantory variabels (s,a,o?,dir,ball_land_x, ball_land_y, x_before, y_before). 
- train model on "df_lagged = pd.read_csv(r"C:\Users\Jenna\OneDrive\Desktop\Statistics\nfl_bdb_neural_net\lagged_2023_w09_game_2023110500.csv")
df_lagged.head()"
- test said neural network on test.csv file they gave us (only has pre release or input data) and submit to kaggle for rmse score 

Remember these are the bounds of the ways we can predict x and y so it needs to be within this:
x: Player position along the long axis of the field, generally within 0-120 yards. (TARGET TO PREDICT)
y: Player position along the short axis of the field, generally within 0 - 53.3 yards. (TARGET TO PREDICT)

kaggle submission info on Submission File:
You must submit to this competition using the provided evaluation API, which ensures that models perform inference on a single play at a time. For each row in the test dataframe you must predict the corresponding x and y values.
Link for sample submission demo: https://www.kaggle.com/code/sohier/nfl-2026-demo-submission/

